In [0]:
import json
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col
from pyspark.sql.functions import substring
import time

In [0]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Exemple de lecture CSV") \
    .getOrCreate()


# Define the schema for the genre DataFrame
genre_schema = StructType([
    StructField("GenreId", IntegerType(), True),
    StructField('Name', StringType(), True)
])

# Define the schema for the artist genre DataFrame
genre_artist_schema = StructType([
    StructField("ID", IntegerType(), True),
    StructField("ArtistID", StringType(), True),
    StructField("GenreID", IntegerType(), True)
])

# Create DataFrame for each defined schema
df_genre = spark.createDataFrame([], schema = genre_schema)
df_genre_artist = spark.createDataFrame([], schema = genre_artist_schema)

# Read CSV file
file_path = '/mnt/raw/genre_data/*.csv'
df_csv = spark.read.csv(file_path, header=True)

# Get current timestamp
start_time = time.time()

#df_csv.show()

# Need to have the list of distinct genre listened
genre_liste = set()

artist_genre_count = 0

# Iterate over each row in the CSV DataFrame
for row in df_csv.rdd.collect():
    artistid = str(row['artist_id'])
    genre = str(row['genres'])

    # I split the genre because it's a list of genre
    genre = genre.strip('[]')
    elements = genre.split(',')
    for elem in elements:
        if elem != '':
            genre_liste.add(elem.strip())
        
genre_dicts = {}
genre_count = 0

# Iterate over each distinct genre
for genre in genre_liste:
    genre_count += 1

    # Create a Spark DataFrame Row
    genre_row = Row(GenreId=genre_count, Name=genre)

    # Append the Row to the genre DataFrame
    df_genre = df_genre.union(spark.createDataFrame([genre_row], schema=genre_schema))

    # To defined an id for the genre
    genre_dicts[genre] = genre_count


# Iterate over each row in the CSV DataFrame
for row in df_csv.rdd.collect():
    artistid = str(row['artist_id'])
    genre = str(row['genres'])
    genre = genre.strip('[]')
    elements = genre.split(',')
    for elem in elements:
        if elem != '':
            artist_genre_count += 1
            
            # Create a Spark DataFrame Row
            artist_genre_row = Row(ID=artist_genre_count,ArtistID=artistid,GenreID=genre_dicts[elem])

            # Append the Row to the genre_artist DataFrame
            df_genre_artist = df_genre_artist.union(spark.createDataFrame([artist_genre_row], schema=genre_artist_schema))

df_genre.show()
df_genre_artist.show()

+-------+--------------------+
|GenreId|                Name|
+-------+--------------------+
|      1|      "chicago soul"|
|      2| "singer-songwriter"|
|      3|       "korean jazz"|
|      4|         "free jazz"|
|      5|       "electronica"|
|      6|"asian american h...|
|      7|         "pop dance"|
|      8|     "neon pop punk"|
|      9|      "classic soul"|
|     10| "post-romantic era"|
|     11|         "kollywood"|
|     12|            "ambeat"|
|     13|             "house"|
|     14|     "rap conscient"|
|     15|     "chicago indie"|
|     16|         "tollywood"|
|     17|               "bgm"|
|     18|"indian undergrou...|
|     19|"afro-cuban percu...|
|     20|       "pop urbaine"|
+-------+--------------------+
only showing top 20 rows

+---+--------------------+-------+
| ID|            ArtistID|GenreID|
+---+--------------------+-------+
|  1|3JsMj0DEzyWc0VDlH...|    196|
|  2|3JsMj0DEzyWc0VDlH...|     50|
|  3|3JsMj0DEzyWc0VDlH...|    144|
|  4|3JsMj0DEzyWc0VD

In [0]:
df_genre.write.mode("overwrite").csv("/mnt/processed/processed-genre")
df_genre_artist.write.mode("overwrite").csv("/mnt/processed/processed-genre-artist")